In [12]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import wget
import requests
from bs4 import BeautifulSoup

### Get nasdaq top 100 companies list

In [23]:
#url = "https://www.nasdaq.com/market-activity/quotes/nasdaq-ndx-index"
#r = requests.get(url)
#s = BeautifulSoup(r.text, "lxml")
dfc = pd.read_csv("data/nasdaq_100_mcap_close.csv")
dfc.head(2)

,symbol,name,market_cap,close
0,ATVI,"Activision Blizzard, Inc. Common Stock","46,69,94,38,722",$60.71
1,ADBE,Adobe Inc. Common Stock,"1,60,62,86,89,302",$332.38


### Get nasdaq top 100 company stock data

In [24]:
def get_company_data(symbol, date_from, date_to):
    url = "https://www.worldtradingdata.com/api/v1/history?symbol="+symbol+"&date_from="+date_from+"&date_to="+date_to+"&sort=newest&api_token=QEs5LtpeTsF7xv85aMmBv5FFl0mQ9BaK3nPZ9ABns4HptPonyZTnoflQfeKy"
    r = requests.get(url)
    if "Message" not in r.json():
        dfs = pd.DataFrame(r.json())
        dfi = pd.DataFrame()
        for i in range(len(dfs)):
            dfi.at[i, "date"] = dfs.index[i]
            dfi.at[i, "close"] = dfs.iloc[i]["history"]["close"]
    else:
        dfi = pd.DataFrame(columns=["date", "close"])
    
    return dfi

In [31]:
dfr = pd.DataFrame()
dfm = pd.DataFrame()
dft = dfc.copy()
#dft = dft[13:]
#dft = dfc[dfc["symbol"] == "TSLA"]

for j in range(len(dft)):
    i = dft.index[j]
    symbol = dft.iloc[j]["symbol"]
    print("Processing company " + str(i) + "/" + str(len(dft)) + " " + symbol)
    date_from="2013-04-01"
    date_to="2020-03-10"
    
    dfi = get_company_data(symbol, date_from, date_to)
    if len(dfi)>0:
        dfi["id"] = i
        dfi["symbol"] = symbol
        #dfi["name"] = dft.iloc[j]["name"]
        dfi = dfi[["id", "symbol", "date", "close"]]

        dfr = dfr.append(dfi)
    else:
        dfi = pd.DataFrame()
        dfi["id"] = i
        dfi["symbol"] = symbol
        #dfi["name"] = dft.iloc[j]["name"]
        dfm = dfm.append(dfi)
        print("No stock data")

    #if j%100==0 and j!=0:
    dfr.to_csv("data/nasdaq_top100_stock_data.csv", index=False, mode="a", header=False)
    dfr = pd.DataFrame()

Processing company 0/103 ATVI
Processing company 1/103 ADBE
Processing company 2/103 AMD
Processing company 3/103 ALXN
Processing company 4/103 ALGN
Processing company 5/103 GOOG
Processing company 6/103 GOOGL
Processing company 7/103 AMZN
Processing company 8/103 AAL
Processing company 9/103 AMGN
Processing company 10/103 ADI
Processing company 11/103 ANSS
Processing company 12/103 AAPL
Processing company 13/103 AMAT
Processing company 14/103 ASML
Processing company 15/103 ADSK
Processing company 16/103 ADP
Processing company 17/103 BIDU
Processing company 18/103 BIIB
Processing company 19/103 BMRN
Processing company 20/103 BKNG
Processing company 21/103 AVGO
Processing company 22/103 CDNS
Processing company 23/103 CDW
Processing company 24/103 CERN
Processing company 25/103 CHTR
Processing company 26/103 CHKP
Processing company 27/103 CTAS
Processing company 28/103 CSCO
Processing company 29/103 CTXS
Processing company 30/103 CTSH
Processing company 31/103 CMCSA
Processing company 32

### Compute annualised volatility 

In [41]:
dfc = pd.read_csv("data/nasdaq_100_mcap_close.csv")

dfr = pd.read_csv("data/nasdaq_top100_stock_data.csv", header=None)
dfr.columns = ["id", "symbol", "date", "close"]
dfr["date"] = pd.to_datetime(dfr["date"])

In [104]:
idx = 0
dfmv = pd.DataFrame()

for i in range(len(dfc)):
    symbol = dfc.iloc[i]["symbol"]
    company_id = dfr[dfr["symbol"] == symbol]["id"].unique()[0]
    name = dfc[dfc["symbol"] == symbol]["name"].values[0]
    mcap = dfc[dfc["symbol"] == symbol]["market_cap"].values[0]
    dfcs = dfr[dfr["symbol"] == symbol].copy()
    dfcs.reset_index(inplace=True)

    prev_close = dfcs["close"][1:]
    prev_close = prev_close.append(pd.Series(np.NAN), ignore_index=True)
    dfcs["prev_close"] = prev_close
    dfcs["change"] = (dfcs["close"] - dfcs["prev_close"]) / dfcs["prev_close"]
    annualised_volatility = np.std(dfcs["change"]) * np.sqrt(252)

    dfmv.at[idx, "id"] = company_id
    dfmv.at[idx, "symbol"] = symbol
    dfmv.at[idx, "name"] = name
    dfmv.at[idx, "mcap"] = mcap
    dfmv.at[idx, "av"] = annualised_volatility
    idx+=1

dfmv["id"] = dfmv["id"].astype(int)
dfmv["mcap"] = dfmv["mcap"].map(lambda x: x.replace(",", ""))
dfmv["av"] = np.round(dfmv["av"], 4)

dfmv.to_csv("data/nasdaq_top100_mcap_av.csv", index=False)    